In [2]:
!pip install -q transformers accelerate huggingface_hub torch sentencepiece

In [3]:
import os
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import logging

# Логирование
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Проверка GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Используемое устройство: {device}")


class Generator:
    """Генерация диалогов через языковую модель"""

    def __init__(self, model_name: str = "sambanovasystems/SambaLingo-Russian-Chat"):
        self.model_name = model_name
        self.model = None
        self.tokenizer = None
        self.device = device

    def load_model(self) -> bool:
        try:
            print("Загрузка модели...")
            self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)
            self.model = AutoModelForCausalLM.from_pretrained(
                self.model_name,
                device_map="auto",
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
            )
            print("✅ Модель загружена.")
            return True
        except Exception as e:
            logger.error(f"Ошибка загрузки модели: {e}")
            return False

    def generate_dialog(self, prompt: str, max_new_tokens=300) -> str:
        if not self.model or not self.tokenizer:
            logger.error("Модель не загружена")
            return ""

        if not prompt.strip():
            logger.error("Промпт пустой")
            return ""

        formatted_prompt = f"<|user|>\n{prompt}\n<|assistant|>\n"

        inputs = self.tokenizer([formatted_prompt], return_tensors="pt").to(self.device)

        with torch.no_grad():
            outputs = self.model.generate(
                **inputs,
                max_new_tokens=max_new_tokens,
                do_sample=True,
                temperature=0.7,
                top_p=0.9,
                repetition_penalty=1.1,
                pad_token_id=self.tokenizer.eos_token_id
            )

        generated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        response = generated_text.split("<|assistant|>")[-1].strip()
        return response

Используемое устройство: cuda


In [4]:
gen = Generator()
if gen.load_model():
    prompt_text = "Диалог между котом и собакой о еде"
    answer = gen.generate_dialog(prompt_text)

    print("\n" + "="*50)
    print(f"Диалог на тему: {prompt_text}")
    print("="*50)
    print(answer)

    # Сохраним в JSON
    with open("result.json", "w", encoding="utf-8") as f:
        json.dump({"prompt": prompt_text, "dialog": answer}, f, ensure_ascii=False, indent=4)
    print("\n✅ Результат сохранён в result.json")

Загрузка модели...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/1.00M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/547 [00:00<?, ?B/s]

...

model-00003-of-00003.safetensors:   0%|          | 0.00/3.98G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/154 [00:00<?, ?B/s]

✅ Модель загружена.

Диалог на тему: Диалог между котом и собакой о еде
Кошка: Привет, пушистый друг! Как проходит твой день?

Собака: Ох, ты знаешь... просто гоняюсь за мячиками и мечтаю о следующем приеме пищи. А вы?

Кошка: Ну, я немного потягиваюсь, почистиваю когти и обдумываю свои гастрономические предпочтения на сегодня.

Собака: Да ладно?! Ты что, серьезно? У меня нет времени думать о еде. Я всегда голоден!

Кошка: (вздыхая) Я знаю, верно? Но когда дело доходит до еды, у нас есть некоторые различия в наших диетических потребностях. В отличие от тебя, кошки — плотоядные животные, а это значит, что мы должны потреблять белок для поддержания нашего тела в тонусе. Поэтому я предпочитаю лакомиться свежим мясом или рыбой – они богаты белком, который помогает наращивать мышцы и поддерживать здоровье кожи и шерсти.

Собака: (перебивая) Подожди минутку, подожди минутку. Разве ты не съедаешь кучу корма для кошек с высоким содержанием белка?

Кошка: Это правда, но важно помнить о типе бел